<a href="https://colab.research.google.com/github/garestrear/poligonal_clusterization/blob/master/poligonal_clusterization_kmeans_with_SSPD_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Poligonal clusterization using kmeans with SSPD distance.

By Gustavo Restrepo. June 2020
##**Problem:**

We want to clusterizate a set of poligonal curves. All of this are standarized in such way that the first point is always (0,0) and the end point is (1,0). 

## **Principal Tools:**
### **1. kmeans**
K-Means is a very simple clustering algorithm (clustering belongs to unsupervised learning). Given a fixed number of clusters and an input dataset the algorithm tries to partition the data into clusters such that the clusters have high intra-class similarity and low inter-class similarity. Here we uses an easy and understandable implementation made for bantak (Git-Hub account of the code author).

#### Algorithm

1. Initialize the cluster centers, either randomly within the range of the input data or (recommended) with some of the existing training examples

2. Until convergence  

   2.1. Assign each datapoint to the closest cluster. The distance between a point and cluster center is measured using diferents distances.  

   2.2. Update the current estimates of the cluster centers by setting them to the mean of all instance belonging to that cluster  
   
   
#### Objective function

The underlying objective function tries to find cluster centers such that, if the data are partitioned into the corresponding clusters, distances between data points and their closest cluster centers become as small as possible.

Given a set of datapoints ${x_1, ..., x_n}$ and a positive number $k$, find the clusters $C_1, ..., C_k$ that minimize

\begin{equation}
J = \sum_{i=1}^n \, \sum_{j=1}^k \, z_{ij} \, || x_i - \mu_j ||_2
\end{equation}

where:  
- $z_{ij} \in \{0,1\}$ defines whether of not datapoint $x_i$ belongs to cluster $C_j$
- $\mu_j$ denotes the cluster center of cluster $C_j$
- $|| \, ||_2$ denotes the Euclidean distance

### Disadvantages of K-Means
- The number of clusters has to be set in the beginning
- The results depend on the inital cluster centers
- It's sensitive to outliers
- It's not guaranteed to find a global optimum, so it can get stuck in a local minimum


###**2. SSPD DISTANCE (Symmetric Segment-Path Distance) [1]:**
This is a modification of the Hausdorff distance. The authos shows a good behaviour of this distance when we are comparing trajectories (or poligonal curves).




  [1] P. Besse, B. Guillouet, J.-M. Loubes, and R. Francois, “Review and perspective for distance based trajectory clustering,” arXiv preprint arXiv:1508.04904, 2015.

# Es necesario instalar la libreria de distancia SSPD

In [1]:
%matplotlib inline
!pip install traj_dist

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Processing /root/.cache/pip/wheels/69/63/0d/560a1741fa3f0ab897105cddb33f21f38c3330e4a57ea75db6/geohash2-1.1-cp36-none-any.whl
  Created wheel for traj-dist: filename=traj_dist-1.15-cp36-cp36m-linux_x86_64.whl size=1496421 sha256=1a657899a6f80db2ac15dc6bf168d4b6134ba97b2c3a6518c304c8f2accbf1fe
  Stored in directory: /root/.cache/pip/wheels/30/da/29/cfefc74eb8b6a6a8b3a1ff2609c988a781d7a7d6c791da5d55
Successfully built traj-dist


### Implementation
We should have to implement kmeans with the new distance SSPD

In [2]:
# %load kmeans.py
import numpy as np
import traj_dist.distance as tdist
import numpy.matlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn import metrics
import scipy.io as spio
import scipy.cluster.hierarchy as sch
import pylab as pl
import pandas as pd
from google.colab import drive
from sklearn.metrics import pairwise_distances, silhouette_samples, silhouette_score


class KMeans:
    
    def __init__(self, n_clusters=100):
        self.K = n_clusters
        
    def fit(self, X):
        #self.centroids = X[np.random.choice(len(X), self.K, replace=False),:]
        self.centroids = X[np.random.choice(len(X), self.K),:]

        self.intial_centroids = self.centroids
        self.prev_label,  self.labels = None, np.zeros(len(X))
        while not np.all(self.labels == self.prev_label) :
            self.prev_label = self.labels
            self.labels = self.predict(X)
            self.update_centroid(X)
        return self
        
    def predict(self, X):
        #return np.apply_along_axis(self.compute_label, 1, X)
        return np.apply_along_axis(self.compute_label, 1, X)


    def compute_label(self, x):
        k = len(self.centroids)
        l=int(np.round(len(x)/2))
        Tc=[]
        for i in range(k):
          ac=np.array(self.centroids[i,:l])
          bc=np.array(self.centroids[i,l:])
          traj=np.transpose(np.array([ac,bc]))
          Tc.append(traj)
        x2=numpy.matlib.repmat(x,k,1)
        Tx=[]
        for i in range(k):
          ax2=np.array(x2[i,:l])
          bx2=np.array(x2[i,l:])
          traj=np.transpose(np.array([ax2,bx2]))
          Tx.append(traj)
        cdist = tdist.cdist(Tc, Tx, metric="sspd")
        ind = np.unravel_index(np.argmin(cdist, axis=None), cdist.shape)
        return ind[0]
        #return np.argmin(tdist.cdist(self.centroids, x, metric="sspd"),axis=1)
    def update_centroid(self, X):
        self.centroids = np.array([np.mean(X[self.labels == k], axis=0)  for k in range(self.K)])

### DATA
We use a database with polygonal curves extracted by discretization of lemniscates, ellipses and hyperbolas. It has 25079 curves.



In [4]:
# Cargamos la base de datos (está en un repositorio en GitHub)
!git  https://github.com/garestrear/poligonal_clusterization.git
mat=spio.loadmat('poligonal_clusterization/DB_N_unif_08_2020.mat',squeeze_me=True)
a=mat['curvasN'] # Es necesario convertirlos a tipo "complex" pues se cargan tipo str
X_numpy=np.array(a,dtype = np.complex) # luego los conver a tipo "array"


git: 'https://github.com/garestrear/poligonal_clusterization' is not a git command. See 'git --help'.


FileNotFoundError: ignored

# Funciones adicionales


*   Función ***dist_pol***: para calcular la distancia de una poligonal a un conjunto
*   Función ***paint_cluster***: para pintar un cluster dado
* Función para calclar la longitud de las poligonales y usarlas para controlar la "semilla" en la clusterización



In [ ]:
## Funciones creadas para este código

# función que calcula la distancia de una poligonal a un conjunto de poligonales 
# almacenadas como las finals de un array
def dist_pol(cpol1, MatPol):
    '''# Esta función calcula la distancia entre una poligonal y una matriz de poligonales
    úsela así [mean_dist, dist]=dist_pol(cpol1,MatPol) y obtenga la distancia promedio de 
    la poligonal a las demás poligonales del cluster y la distancia mínima '''
    r=MatPol-cpol1
    d2=np.apply_along_axis(np.linalg.norm,1,r)
    dist=min(d2)
    mean_dist=np.mean(d2)
    return mean_dist, dist
# Definimos un afunción que pinta un custer
def paint_cluster(ncluster,clusters_pack):
  '''# Use esta función así paint_cluster(ncluster,clusters_pack)
  # recuerde que para obtener clusters_pack use
  # clusters_pack= [X_numpy[labels==i] for i in range(cluster_number)]'''
  X_numpy_C=clusters_pack[ncluster]
  X=X_numpy_C.real
  Y=X_numpy_C.imag
  # Pintamos
  plt.axis('equal')
  plt.plot(np.transpose(X),np.transpose(Y))
  plt.suptitle(('Cluster number %d ' % ncluster, len(X)),fontsize=28, fontweight='bold')

def longitudes_pol(Mat_pol):
    long_pol=list()
    for pol in Mat_pol:
        l=np.sum(np.abs(pol[0:-1]-pol[1:]))
        long_pol.append(l)
    return long_pol



# Representation of the Database
We choose in an aleatory way some polygonals to paint

In [ ]:
# Pintamos algunas de esas poligonales
N_curvas2paint=400;
a=np.random.randint(0,len(X_numpy), N_curvas2paint)
b=np.sort(a)
#print(b)
longitudes=longitudes_pol(X_numpy)
X=X_numpy.real
Y=X_numpy.imag
for k in b:
  plt.plot(X[k,0:15],Y[k,0:15])
  plt.axis('equal')
plt.show()

# Reorganizamos las curvas según su longitud


In [ ]:
longitudes2=np.array(longitudes)
longitudes3 =np.sort(longitudes2)
sort_indices=np.argsort(longitudes2)
X_numpy_sort=X_numpy[sort_indices,:]
plt.hist(longitudes3, bins=1000)
plt.xlim([0,2])
print('La longitud máxima es: '+ str(np.max(longitudes3)))
plt.show()

Según el histograma anterior un gran número de poligonales tienen longitudes entre 1 y 1.5. Por lo anterior no es conveniente incluir las poligonales con mayor longitud en el proceso de clusterización. En palabras coloquiales estariamos introduciendo "ruido".

Clusterizamos el 95% de las curvas. 


In [ ]:
a=0.95*len(X_numpy_sort)
indice_final=int(np.floor(a))
X_numpy_short=X_numpy_sort[range(indice_final),:]
# Hacemos de nuevo el histograma
longitudes4 =longitudes3[range(indice_final)]
plt.hist(longitudes4, bins=1000)
plt.ylim([0,800])
print('La longitud máxima es: '+ str(np.max(longitudes4)))
plt.show()

# Clusterization
Now we have the ingredients for the clusterization process. Lets make it real!

In [ ]:
# Iniciamos el proceso de Clusterización
np.random.seed(5)
cluster_number=20
X_numpy_r30_2=np.concatenate((X_numpy_sort.real, X_numpy_sort.imag),axis=1)
poligonal_number=500
X_numpy_r30=X_numpy_r30_2[0:poligonal_number,:]
#longitudes=longitudes4[0:poligonal_number]
#X=X_numpy_r30
kmeans_model =KMeans(n_clusters=cluster_number) # estimators=[('k_means_iris_150', KMeans(n_clusters=150))]
kmeans_model.fit(X_numpy_r30)
labels = kmeans_model.labels
#fignum = 1
#titles = ['kmeans + SSPD with 150 custers']
#estimators.fit(X)
kmeans_Silhuette_coefficient=metrics.silhouette_score(X_numpy_r30,labels,metric='euclidean')

print(labels)

## **Cálculo de la cantidad de curvas en cada cluster y reorganización de la clustrización**


In [ ]:
labels2=labels
# algoritmo para reorganización de clusters
labels_aux=labels
X_numpy2=X_numpy[0:poligonal_number,:]
for k in range(cluster_number-1):
  for j in range(k+1,cluster_number):
    Numk=len(X_numpy2[labels_aux==k])
    Numj=len(X_numpy2[labels_aux==j])
    if Numk < Numj:
      a=labels_aux==j
      b=labels_aux==k
      labels_aux[a]=k
      labels_aux[b]=j
         
print(labels_aux)


# Cluster Visualization. 
Use la función 

 paint_cluster(ncluster,clusters_pack)

 Para pintar un cluster, 
 Haga uso de la siguiente celda

In [ ]:
from pylab import *
clusters_pack2 = [X_numpy2[labels_aux==i] for i in range(cluster_number)]
for f in range(cluster_number):
  figure(f)
  #ncluster=0 # Ponga el número del cluster que desea pintar
  paint_cluster(f,clusters_pack2)
show()


# Silhuette analisis
The Silhouette Coefficient is a measure of how well samples are clustered with samples that are similar to themselves. Clustering models with a high Silhouette Coefficient are said to be dense, where samples in the same cluster are similar to each other, and well separated, where samples in different clusters are not very similar to each other.

The Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample. The Silhouette Coefficient for a sample is (b - a) / max(a, b). To clarify, b is the distance between a sample and the nearest cluster that the sample is not a part of. Note that Silhouette Coefficient is only defined if number of labels is 2 <= n_labels <= n_samples - 1.
Explicación:
Suponga que se ha realizado una clusterización sobre la base de datos.
El silhouette coeff. (sc_i) se calcula para cada poligonal. Así:
Suponga que tomamos la poligonal i-ésima y que está en el cluster #k (Ck)
Paso 1. Se halla el promedio de las distancias de la poligonal i, a todas las poligonales contenidas en el mismo cluster (cluster Ck). Este número se denota por (a_i), i= desde 1 hasta #poligonales.

Paso 2. Se halla la distancia promedio de la poligonal i a las poligonales del cluster Cj, con j diferente de k y luego se toma el mínimo de estas distancias promedios. Este número se denota (b_i)
Observaciones.
Se espera que b_i > a_i. Caso contrario la poligonalestá mal ubicada (Hay un cluster en el que encajaría mejor).
El silhuette coeff (sc_i), es: (b_i-a_i)/max(b_i,a_i).
El denominador max(b_i,a_i) es un factor de normalización.
El caso ideal sería: todos los cluster cumplen que la distancia entre dos de sus elementos es siempre cero y por tanto a_i=0 y la distancia entre elementos de clusters diferentes es siempre mayor que cero. En este caso sc_i=1.
El peor caso es cuando dos elementos iguales quedan en diferente cluster y por tanto b_i=0 pero la distancia entre elementos del mismo cluster no es cero así a_i>0. En este caso sc_i=-1.
Finalmente el silhuette coeff. (sc) de la clusterización es (b-a)/max(a,b) donde b es promedio de los b_i y a es el promedio de los a_i.


The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.



## Silhuette coefficient for each cluster


### Sihuette coefficients for kmeans

In [ ]:
# Arriba se calculó
#hc_Silhuette_coefficient=metrics.silhouette_score(X_numpy_r30, labels, metric='euclidean')
kmeans_Silhuette_coefficient=metrics.silhouette_score(X_numpy_r30, labels_aux, metric='euclidean')

# El anterior número da cuenta de la calidad de la clusterización
silhouette_avg=kmeans_Silhuette_coefficient

# Compute the silhouette scores for each sample
sample_silhouette_values = silhouette_samples(X_numpy_r30, labels_aux)
#print(sample_silhouette_values)
fig, ax1 = plt.subplots(1, 1)
fig.set_size_inches(36, 100)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
ax1.set_xlim([-0.4, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
space_cluster=75
ax1.set_ylim([0, len(X_numpy2) + (cluster_number + 1) * space_cluster])
y_lower = space_cluster
color_option=['green', 'blue']
for i in range(cluster_number):
    # Aggregate the silhouette scores for samples belonging to
    # cluster i, and sort them
    ith_cluster_silhouette_values = \
      sample_silhouette_values[labels_aux == i]
    
    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    
  
    #color = cm.nipy_spectral(float(i) / cluster_number)  #Colormap type
    color=color_option[i % 2]
    ax1.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_values,
                      facecolor=color, edgecolor=color, alpha=0.7)
    # Pintamos también el promedio de los coeficientes silhuette en cada cluster
    #ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    mpc=np.mean(sample_silhouette_values[labels_aux == i]) #mean per cluster
    #ax1.axvline(x=mpc, ymin=y_lower, ymax=y_lower+size_cluster_i, color="red", linestyle="--")
    ax1.plot([mpc, mpc],[y_lower, y_lower+size_cluster_i], \
             color="green", linestyle="--",linewidth=4)
   
   # Label the silhouette plots with their cluster numbers at the middle
    if size_cluster_i < 5:
      if i % 2 == 1: # para escribir solo los impares
        ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=20)
        ax1.plot([mpc],[y_lower],'og')
      else:
        ax1.plot([mpc],[y_lower],'og')
    elif size_cluster_i < 10:
      if i % 2 == 1: # para escribir solo los impares
        ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=20)
        ax1.plot([mpc],[y_lower],'og')
      else:
        ax1.plot([mpc],[y_lower],'og')
    elif size_cluster_i < 20:
      if i % 2 == 1: # para escribir solo los impares
        ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=20)
        ax1.plot([mpc],[y_lower],'og')
      else:
        ax1.plot([mpc],[y_lower],'og')
    elif size_cluster_i < 50:
      if i % 2 == 1: # para escribir solo los impares
        ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=20)
        ax1.plot([mpc],[y_lower],'og')
      else: 
        ax1.plot([mpc],[y_lower],'og') 
    elif size_cluster_i < 100:
      ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=20)
    elif size_cluster_i < 500:
      ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=27)
    elif size_cluster_i < 1000:
      ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=35)
    else:
      ax1.text(-0.2, y_lower + 0.5 * size_cluster_i, 'C'+str(i)+' with '+str(size_cluster_i),size=40)

    # Compute the new y_lower for next plot
    y_lower = y_upper + space_cluster  # 10 for the 0 samples

ax1.set_title("The silhouette plot for the various clusters.",fontsize=36)
ax1.set_xlabel("The silhouette coefficient values",fontsize=36)
ax1.set_ylabel("Cluster label",fontsize=36)

    # The vertical line for average silhouette score of all the values
ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

ax1.set_yticks([])  # Clear the yaxis labels / ticks
ax1.set_xticks([-0.4,-0.3,-0.2,-0.1, 0, 0.1,0.2, 0.3,0.4,0.5, 0.6, 0.7,0.8,\
                    0.9, 1])
#plt.xticks(fontsize=16)
plt.xticks(fontsize=34)
plt.suptitle(("Silhouette analysis for kmeans clusterization "
               "with n_clusters = %d" % cluster_number),
              fontsize=28, fontweight='bold')

 #plt.show()
   
